# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../output_data/weather_SP.csv"
data_df = pd.read_csv(file_path)
data_df

,City,Lat,Lng,Country,Date,Max Temp,Humidity,Wind Speed,Cloudiness,Max Temp (F)
0,torbay,47.67,-52.73,CA,1587336766,1.67,92.0,14.90,90.0,35.006
1,naze,28.37,129.48,JP,1587337038,19.00,77.0,4.60,75.0,66.200
2,cabo san lucas,22.89,-109.91,MX,1587337038,27.00,42.0,5.70,5.0,80.600
3,avarua,-21.21,-159.78,CK,1587337038,29.00,66.0,5.10,2.0,84.200
4,hirara,24.80,125.28,JP,1587337038,24.00,94.0,4.60,75.0,75.200
...,...,...,...,...,...,...,...,...,...,...
548,bloomington,44.84,-93.30,US,1587337110,10.56,24.0,3.10,20.0,51.008
549,korla,41.76,86.15,CN,1587337110,14.44,45.0,2.24,100.0,57.992
550,lamar,33.67,-95.58,US,1587337110,17.78,100.0,2.60,90.0,64.004
551,majene,-3.54,118.97,ID,1587337110,27.20,79.0,1.42,97.0,80.960


In [3]:
data_df["Humidity"].max()

100.0

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = data_df[["Lat", "Lng"]]
humidity = data_df["Humidity"]

fig1 = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False,max_intensity=100,
                                 point_radius=3)

fig1.add_layer(heat_layer)

fig1


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather = data_df.loc[(data_df["Max Temp (F)"]>=70) & (data_df["Max Temp (F)"]<80) & (data_df["Wind Speed"] < 10) & (data_df["Cloudiness"] < 10),]
ideal_weather 

,City,Lat,Lng,Country,Date,Max Temp,Humidity,Wind Speed,Cloudiness,Max Temp (F)
5,east london,-33.02,27.91,ZA,1587336763,24.56,54.0,5.97,0.0,76.208
34,cockburn town,21.46,-71.14,TC,1587336816,26.21,82.0,7.37,7.0,79.178
40,chimore,-16.98,-65.13,BO,1587336890,22.23,57.0,1.05,1.0,72.014
61,kapaa,22.08,-159.32,US,1587337045,26.00,65.0,3.60,1.0,78.800
68,aswan,24.09,32.91,EG,1587336822,21.39,30.0,3.73,0.0,70.502
108,pimentel,-6.84,-79.93,PE,1587337052,22.29,78.0,5.55,4.0,72.122
112,saint george,37.10,-113.58,US,1587336796,22.78,28.0,6.70,1.0,73.004
123,tahta,26.77,31.50,EG,1587337054,24.00,38.0,2.10,0.0,75.200
176,beloha,-25.17,45.05,MG,1587336851,21.62,76.0,7.16,0.0,70.916
219,sao filipe,14.90,-24.50,CV,1587337066,22.12,74.0,8.81,0.0,71.816


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather.copy()
hotel_df["Hotel Name"] = ""
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 5 to 520
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   City          18 non-null     object 
 1   Lat           18 non-null     float64
 2   Lng           18 non-null     float64
 3   Country       18 non-null     object 
 4   Date          18 non-null     int64  
 5   Max Temp      18 non-null     float64
 6   Humidity      18 non-null     float64
 7   Wind Speed    18 non-null     float64
 8   Cloudiness    18 non-null     float64
 9   Max Temp (F)  18 non-null     float64
 10  Hotel Name    18 non-null     object 
dtypes: float64(7), int64(1), object(3)
memory usage: 1.7+ KB


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    params = {"location": str(row["Lat"]) + ", " + str(row["Lng"]), "types": "hotel", "radius": 5000, "key": g_key}
    response = requests.get(base_url, params).json()
    hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    


In [9]:
hotel_df      

,City,Lat,Lng,Country,Date,Max Temp,Humidity,Wind Speed,Cloudiness,Max Temp (F),Hotel Name
5,east london,-33.02,27.91,ZA,1587336763,24.56,54.0,5.97,0.0,76.208,East London
34,cockburn town,21.46,-71.14,TC,1587336816,26.21,82.0,7.37,7.0,79.178,Cockburn Town
40,chimore,-16.98,-65.13,BO,1587336890,22.23,57.0,1.05,1.0,72.014,Chimoré
61,kapaa,22.08,-159.32,US,1587337045,26.00,65.0,3.60,1.0,78.800,Kapaʻa
68,aswan,24.09,32.91,EG,1587336822,21.39,30.0,3.73,0.0,70.502,Aswan
108,pimentel,-6.84,-79.93,PE,1587337052,22.29,78.0,5.55,4.0,72.122,Cercado de Pimentel
112,saint george,37.10,-113.58,US,1587336796,22.78,28.0,6.70,1.0,73.004,St. George
123,tahta,26.77,31.50,EG,1587337054,24.00,38.0,2.10,0.0,75.200,Tahta
176,beloha,-25.17,45.05,MG,1587336851,21.62,76.0,7.16,0.0,70.916,Beloha
219,sao filipe,14.90,-24.50,CV,1587337066,22.12,74.0,8.81,0.0,71.816,Sao Filipe


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]
hotel_locations

,Lat,Lng
5,-33.02,27.91
34,21.46,-71.14
40,-16.98,-65.13
61,22.08,-159.32
68,24.09,32.91
108,-6.84,-79.93
112,37.10,-113.58
123,26.77,31.50
176,-25.17,45.05
219,14.90,-24.50


In [11]:
fig1

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(hotel_locations)

fig1.add_layer(markers)
fig1

# Display Map

Figure(layout=FigureLayout(height='420px'))